# How do I use dsco? (commands)

Here we will do an introduction to dsco commands. 

To do this we will be setting up a new project called `foo`.

## Setup

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

Create a working directory:

In [2]:
!mkdir -p /tmp/dsco_overview

In [3]:
cd /tmp/dsco_overview/

/private/tmp/dsco_overview


## --help

We can use `--help` to see a list of the available commands and a 
short description of what they do.

In [4]:
!dsco --help

usage: dsco [-h]
            {build,code,documentation,go,init,ls,reports,restart,rm,rmi,shell,up,update_port}
            ...

positional arguments:
  {build,code,documentation,go,init,ls,reports,restart,rm,rmi,shell,up,update_port}
                        available functionality
    build               build images. No flag builds dev.
    code                launch vs-code
    documentation       view and update documentation in the dev container
    go                  Build containers, then open the web page interface.
    init                create a new project
    ls                  list containers
    reports             view and update reports in the dev container
    restart             restart nginx and flask (dev)
    rm                  remove containers
    rmi                 remove images
    shell               start shell in container
    up                  create and launch containers
    update_port         update container port (container restart required)

opti

## --init

In [5]:
!dsco init --help

usage: dsco init [-h]

optional arguments:
  -h, --help  show this help message and exit


The first thing we'll want to do is create our project. 
We do this with `--init`

Normally the `--init` command prompts us for input. However this is being 
done in a jupyter notebook, which does not allow for interactive 
responses. To get around this we will pipe in our responses as follows:
    
    - project_name [MyProject]: foo
    - author [Discovery]: 
    - version [0.1]: 
    - year [2020]: 
    - project_port[8001]: 
    
The only default we're changing is the project name. In actual usage 
just run `dsco init` and answer the prompts.

In [6]:
!echo "foo\n\n\n\n\n" | dsco init

Using template: /Users/ap186098/.pyenv/versions/3.7.5/lib/python3.7/site-packages/dsco/project_template
Creating project in /private/tmp/dsco_overview
project_name [MyProject]: author [Discovery]: version [0.1]: year [2020]: project_port [8001]: 

### PROJECT DIRECTORY

In [7]:
cd foo

/private/tmp/dsco_overview/foo


In [8]:
!ls

README.md          definitions.py     flask_config.ini   landing_page
__pycache__        docker-compose.yml flask_run.py       pyproject.toml
builder            docs               foo                tests


One of the nice things about using dsco to create your project is the 
consistency in project structure. Let's point out a few of the key 
features.

#### README.md

A standard project README file.

#### builder

The builder directory contains everything necessary to build the project 
containers including docker files and ansible playbooks.

#### definitions.py

In [9]:
cat definitions.py

#!/usr/bin/env python
# * encoding: utf-8
import os


def _test_dir(dir_path):
    if not os.path.isdir(dir_path):
        raise IOError(f"Expected specs directory at {dir_path}")


ROOT_DIR = os.path.dirname(os.path.abspath(__file__))


SQL_DIR = os.path.join(ROOT_DIR, "foo", "sql")
_test_dir(SQL_DIR)


This is a placeholder for defining a few useful project variables. 
If you need access to the project root directory you can import 
it from definitions.

#### docker-compose.yml

This defines the service definitions: dev, prod, and debug for use 
with docker-compose. This is where they are tied to the appropriate 
docker file.

#### docs

In [10]:
!tree docs

docs
├── assets
│   ├── banner.png
│   ├── space-shuttle-launch.png
│   └── sphinxheader.png
├── documentation
│   ├── html
│   │   └── plotly.js
│   └── source
│       ├── Makefile
│       ├── __pycache__
│       │   └── conf.cpython-37.pyc
│       ├── _static
│       │   └── plotly.js
│       ├── conf.py
│       └── index.rst
├── index.html
└── reports
    ├── html
    │   └── plotly.js
    └── source
        ├── Makefile
        ├── __init__.py
        ├── __pycache__
        │   ├── __init__.cpython-37.pyc
        │   └── conf.cpython-37.pyc
        ├── _static
        │   └── plotly.js
        ├── components.png
        ├── conf.py
        ├── index.rst
        ├── reports_template.ipynb
        └── resources.ipynb

11 directories, 21 files


The docs directory is important. Dsco distinguishes between two different 
types of docs: documentation and reports.

Documentation is project documentation. The how and why of your code.

Reports is meant to be a place to put your exploratory analysis.

Dsco facilitates the production of a static html representation of each of 
these types of documentation. Put any .rst or notebook files in the source 
directory and generate html with either `dsco documentation -g` or 
`dsco reports -g`.

#### flask_config.ini

In [11]:
cat flask_config.ini

[uwsgi]
socket = 127.0.0.1:3031
mount = /webapp=flask_run.py
callable=app

master = true
processes = 2

chmod-socket = 660
vacuum = true

die-on-term = true
manage-script-name = true



The container is running Flask with uwsgi. This is the configuration file. 
Don't mess with it unless you know what you're doing.

#### flask_run.py

In [12]:
cat flask_run.py

"""App entry point."""
from foo import create_app

app = create_app()

if __name__ == "__main__":
    app.run(host="0.0.0.0", debug=True)


This is the flask entry point. Again, don't mess with it unless you 
know what you're doing.

#### project_name (e.g. foo)

In [13]:
!tree foo

foo
├── __init__.py
├── __pycache__
│   ├── __init__.cpython-37.pyc
│   ├── connection.cpython-37.pyc
│   ├── flask_main.cpython-37.pyc
│   ├── foo.cpython-37.pyc
│   ├── helpers.cpython-37.pyc
│   ├── queries.cpython-37.pyc
│   └── routes.cpython-37.pyc
├── connection.py
├── dashapp1
│   ├── __init__.py
│   └── __pycache__
│       └── __init__.cpython-37.pyc
├── flask_main.py
├── foo.py
├── helpers.py
├── queries.py
├── routes.py
└── sql
    ├── all_test_queries.sql
    └── events.sql

4 directories, 18 files


This is your python package directory. It will have whatever name you 
provided in the `--init` command for `[project_name]`. This is a standard 
setup for creating python packages.

If you're not creating a package, don't worry about it. It's not taking 
up a lot of overhead. Just create notebooks in the `docs/reports/source` 
directory and leave this alone.

If you are creating a package, there are some nice features in this package 
directory you should know about.

> **Opinionated perspective**
> 
> In the introduction, we said we take an opinionated perspective on the 
> data science development perspective. That started with the first step 
> of doing EDA in jupyter notebooks. 
> 
> After EDA, the next level of maturity is to build some sort of dashboard. 
> Dsco sets up the scaffolding for using plotly | Dash in conjunction with 
> Flask to build your dashboard.

The flask app is created in `flask_main.py`. Flask routes are 
configured in `routes.py`. This is where you can create 
rest API routes that would return data. 

The dash app is  created in `dashapp1/__init__.py`.

`nginx` is configured to handle requests to `/`. Nginx also handles serving 
the static content generated in `docs`. This can be found at 
`/reports/` and `/documentation/`.

`flask` is configured to handle requests to `/webapp/`.

`dash` is configured to handle requests to `/webapp/dash`.


#### landing_page

The landing_page directory contains the html for the page you see 
when you connect to the container at `http://<container>:<port>/`. 

#### pyproject.toml, poetry.lock

These files manage our python dependencies. For more information see 
http://https://python-poetry.org/.

#### tests

TODO: At the moment this is more of a place holder. 
The intention is to build scaffolding around testing and wrap 
that in something like `dsco test`. 

## --build

Now that we have our project directory, it's time to build our 
development image. We do that with `dsco build`.

In [14]:
!dsco build --help

usage: dsco build [-h] [--dev] [--prod] [--debug] [--all]

optional arguments:
  -h, --help  show this help message and exit
  --dev       build dev image
  --prod      build prod image
  --debug     build debug image
  --all       build all images


In [15]:
# this produces a lot of output from ansible and takes ~8 minutes
!dsco build

docker-compose build dev
Building dev
Step 1/8 : FROM python:3.7.5-slim-buster as base
 ---> 07ee12a5eb2a
Step 2/8 : COPY . /srv
 ---> fe7bea56aca8
Step 3/8 : SHELL ["/bin/bash", "-c"]
 ---> Running in 979b9edc0383
Removing intermediate container 979b9edc0383
 ---> 54b5f1dddb06
Step 4/8 : RUN pip install ansible toml
 ---> Running in dcbc0ef6d114
  Created wheel for ansible: filename=ansible-2.9.6-cp37-none-any.whl size=16164186 sha256=6e0bd6c2ef9e37e73be07f049833a55e5f0c615ed06f0b582f6d1334bb189db6
  Stored in directory: /root/.cache/pip/wheels/e0/d8/74/a4e0b92e45a38ab0c22eaadc5981f3ddab41a6e188c0c8867d
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=b4f815b4dd4db5f08317f85d97d4eba20aed66ec1dc880d3a2159c843665280d
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built ansible PyYAML
You should consider upgrading via the 'pip install --upgrade pip' command.
Removing in

changed: [localhost]

TASK [jupyter-config : .jupyter directory] *************************************
changed: [localhost]

TASK [jupyter-config : jupyter_notebook_config.py] *****************************
changed: [localhost]

TASK [jupyter-config : nbconfig directory] *************************************
changed: [localhost]

TASK [jupyter-config : notebook.json] ******************************************
changed: [localhost]

TASK [jupyter-config : custom directory] ***************************************
changed: [localhost]

TASK [jupyter-config : custom.js] **********************************************
changed: [localhost]

TASK [jupyter-config : install nbextensions] ***********************************
changed: [localhost]

TASK [jupyter-config : copy jupyter.conf file to supervisor dir] ***************
changed: [localhost]

TASK [jupyter-config : update supervisor] **************************************
changed: [localhost]

TASK [sphinx-config : Check if sphinx is installed]

Generally this build takes about 8 minutes (see execution time at the end).

We can see that it built if we run `dsco ls`. We should see 
`foo_dev:latest` listed.

In [16]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     3 seconds ago                 1.75GB    


## --up

The next step is to create a container from the image. If the image is 
already built, this is really fast, if the image has not been built, 
this will first build the image, and then create the container, which 
will take about 8 minutes.

In [17]:
!dsco up

docker-compose up -d dev
Creating foo_dev_1 ... 
ting foo_dev_1 ... done
up execution time: 1.7827799320220947 seconds


We can use `dsco ls` to see if the container is up.

In [18]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     7 seconds ago                 1.75GB    
    foo_dev_1                     http://localhost:8001                      cca408b97e0a     Up Less than a second                   


We can access the container at the specified link. If the container is not 
running you will see this.

In [19]:
# we can use docker commands to interact with our containers as well
# this one stops the container without deleting it
!docker stop foo_dev_1

foo_dev_1


In [20]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     8 seconds ago                 1.75GB    
    foo_dev_1                     http://_________:8001                      cca408b97e0a     Exited (143) Less than a second ago          


In [21]:
# start it again ...
!dsco up

docker-compose up -d dev
Starting foo_dev_1 ... 
ting foo_dev_1 ... done
up execution time: 1.3186349868774414 seconds


## --go

In [22]:
!dsco go --help

usage: dsco go [-h] [--dev] [--prod] [--debug] [--all]

optional arguments:
  -h, --help  show this help message and exit
  --dev       dev container
  --prod      prod container
  --debug     debug container
  --all       all containers


Here we have used `dsco build` and `dsco up` to build and launch our 
dev container. We also could have used `dsco go`. 

`dsco go` runs `dsco up` and then launches a web browser connected 
to the container. If a container is already running, the only 
effect will be to open a web browser.

In [23]:
!dsco go

docker-compose up -d dev
foo_dev_1 is up-to-date

up execution time: 0.9284379482269287 seconds

go execution time: 1.2834627628326416 seconds


## --documentation / --reports

If we try to go to access our reports or documentation through the links 
on the landing page we will be met with an error 403 message. This is 
because we have not built it yet.

In [24]:
# build documentation
!dsco documentation --generate

docker exec -it $(docker ps --all --filter 'label=com.docker.compose.project=foo' --filter 'label=com.docker.compose.service=dev' --format {{.Names}}) bash -c 'cd /srv/docs/documentation/source && make html'
Running Sphinx v2.4.4
building [mo]: targets for 0 po files that are out of date
building [html]: targets for 1 source files that are out of date
updating environment: [new config] 1 added, 0 changed, 0 removed
reading sources... [100%] index                                                 
looking for now-outdated files... none found
pickling environment... done
checking consistency... done
preparing documents... done
/usr/local/lib/python3.7/site-packages/guzzle_sphinx_theme/guzzle_sphinx_theme/layout.html:200: RemovedInSphinx30Warning: To modify script_files in the theme is deprecated. Please insert a <script> tag directly in your theme instead.

generating indices...  genindexdone
writing additional pages...  searchdone
copying static files... ... done
copying extra files... do

In [25]:
# build reports
!dsco reports --generate

docker exec -it $(docker ps --all --filter 'label=com.docker.compose.project=foo' --filter 'label=com.docker.compose.service=dev' --format {{.Names}}) bash -c 'cd /srv/docs/reports/source && make html'
Running Sphinx v2.4.4
building [mo]: targets for 0 po files that are out of date
building [html]: targets for 3 source files that are out of date
updating environment: [new config] 3 added, 0 changed, 0 removed
reading sources... [100%] resources                                             
/srv/docs/reports/source/reports_template.ipynb:24: WARNING: File not found: '/notebook/notebooks/reports/reports_template.ipynb'
/srv/docs/reports/source/reports_template.ipynb:24: WARNING: File not found: '/reports/reports_template.html#'
/srv/docs/reports/source/reports_template.ipynb:41: WARNING: File not found: '/notebook/edit/reports/index.rst'
looking for now-outdated files... none found
pickling environment... done
checking consistency... /srv/docs/reports/source/resources.ipynb: WARNING: docu

Now we should be able to access the starting points of our documentation and reports.

Of course, there's just some basic scaffolding in place. To create the documentation 
write some content in reStructuredText files or jupyter notebooks and add these files 
to `index.rst` in the `source/` directory.

As a simple example:

In [26]:
%%writefile docs/reports/source/example.rst
Example
=======

This is an example .rst file

Writing docs/reports/source/example.rst


In [27]:
%%writefile docs/reports/source/index.rst
.. Template documentation master file, created by
   sphinx-quickstart
   You can adapt this file completely to your liking, but it should at least
   contain the root `toctree` directive.

Welcome to foo's documentation!
===============================

.. toctree::
   :maxdepth: 2
   :caption: Contents:

   reports_template
   example


Indices and tables
==================

* :ref:`genindex`
* :ref:`modindex`
* :ref:`search`


Overwriting docs/reports/source/index.rst


In [28]:
!dsco reports --generate

docker exec -it $(docker ps --all --filter 'label=com.docker.compose.project=foo' --filter 'label=com.docker.compose.service=dev' --format {{.Names}}) bash -c 'cd /srv/docs/reports/source && make html'
Running Sphinx v2.4.4
loading pickled environment... done
building [mo]: targets for 0 po files that are out of date
building [html]: targets for 1 source files that are out of date
updating environment: 1 added, 1 changed, 0 removed
reading sources... [100%] index                                                 
looking for now-outdated files... none found
pickling environment... done
checking consistency... /srv/docs/reports/source/resources.ipynb: WARNING: document isn't included in any toctree
done
preparing documents... done
/usr/local/lib/python3.7/site-packages/guzzle_sphinx_theme/guzzle_sphinx_theme/layout.html:200: RemovedInSphinx30Warning: To modify script_files in the theme is deprecated. Please insert a <script> tag directly in your theme instead.
writing output... [100%] ind

The rendering isn't perfect here, but you can see a link to `Example` 
under the `Welcome to foo's documentation!` section.

In [29]:
display(HTML("docs/reports/html/index.html"))

And another imperfect rendering of the `Example` page we created.

In [30]:
display(HTML("docs/reports/html/example.html"))

## --ls

In [31]:
!dsco ls --help

usage: dsco ls [-h] [-r]

optional arguments:
  -h, --help    show this help message and exit
  -r, --remote  list remote containers using docker over ssh


We've already been using `dsco ls` to see the containers and images that 
are present on our local machine.

We can also use `dsco ls -r` to see containers we have running on remote 
machines. This leverages docker's ability to run docker commands over 
ssh by setting the `DOCKER_HOST` environment variable.

For this to work, you need to have keyless ssh set up to your remote host, 
and then create a `$HOME/.dsco/settings.json` file that contains 
the following:

```
{
    "remote": [
        {
            "name": "my_name_for_my_machine",
            "properties": {
                "ip": "fully_qualified_domain_name"
            },
            "env": {
                "DOCKER_HOST": "ssh://user@fqdn"
            }
        },
        {
            "name": "my_name_for_my_machine2",
            "properties": {
                "ip": "fully_qualified_domain_name"
            },
            "env": {
                "DOCKER_HOST": "ssh://user@fqdn"
            }
        }
    ]
}
```

In [32]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     28 seconds ago                1.75GB    
    foo_dev_1                     http://localhost:8001                      cca408b97e0a     Up 17 seconds                           


## --shell

In [33]:
!dsco shell --help

usage: dsco shell [-h] [--dev] [--prod] [--debug]

optional arguments:
  -h, --help  show this help message and exit
  --dev       attach to dev
  --prod      attach to prod
  --debug     attach to debug


Use the `dsco shell` command to open a bash shell inside your container.

This is the preferred way to work with your project dependencies.

## --code

In [34]:
!dsco code --help

usage: dsco code [-h] [-r REMOTE]

optional arguments:
  -h, --help            show this help message and exit
  -r REMOTE, --remote REMOTE
                        connect to a container with docker-machine


Use `dsco code` to launch a session of Microsoft's vscode IDE. After you 
install the `Remote Development Extension`, you can use the 
`Remote Explorer` to attach to running containers. Open up the `/srv` 
directory and enjoy a "local-like" development experience.

Even more amazing is the ability to attach to containers running on 
remote machines. If you have remote machines set up as described in the 
`--ls` section, you can simple type `dsco code -r my_name_for_my_machine`.
When vscode opens you can use `Remote Explorer` to attach to the listed 
containers, but this time you will see the containers running on the 
remote machine. Super Cool!!!

You can read more about using vscode with containers 
here: https://code.visualstudio.com/docs/remote/containers.

## --restart

In [35]:
!dsco restart --help

usage: dsco restart [-h] [--flask] [--nginx]

optional arguments:
  -h, --help  show this help message and exit
  --flask     restart flask
  --nginx     restart nginx


When you're working on creating your Dash dashboard, you will need to 
restart flask to see the changes you made. `dsco restart` is an easy 
way to do this. Without options it will restart both `nginx` and `flask`. 
Use the flags to be more specific.

In [36]:
!dsco restart

docker exec -it $(docker ps --all --filter 'label=com.docker.compose.project=foo' --filter 'label=com.docker.compose.service=dev' --format {{.Names}}) bash -c "service nginx restart"
[....] Restarting nginx: nginx7[ ok 8.
docker exec -it $(docker ps --all --filter 'label=com.docker.compose.project=foo' --filter 'label=com.docker.compose.service=dev' --format {{.Names}}) bash -c "supervisorctl restart uwsgi && supervisorctl status"
uwsgi: stopped
uwsgi: started
jupyter                          RUNNING   pid 57, uptime 0:00:21
uwsgi                            RUNNING   pid 445, uptime 0:00:01


## --update_port

In [37]:
!dsco update_port --help

usage: dsco update_port [-h] [--dev DEV] [--prod PROD] [--debug DEBUG]
                        [base_port]

positional arguments:
  base_port      update ports dev: base_port, prod: base_port+1000, debug:
                 base_port-500

optional arguments:
  -h, --help     show this help message and exit
  --dev DEV      update dev port, overwrites base_port
  --prod PROD    update prod port, overwrites base_port
  --debug DEBUG  update debug port, overwrites base_port


When you set up your project, you were prompted for a port number. By 
default, this is the port of the `dev` container. The `prod` container 
is assigned `port + 1000` and the `debug` container is assigned `port - 500`.

If you're running multiple containers on the same machine, they all need 
to be on different ports. To change the port you can use `dsco update_port`

If you enter `dsco update_port 8001`: you will get the default behavior 
described above. If you want to override this behavior, simply add the 
appropriate flag and specify the desired port.

For example:
```
    dsco update_port 8001 --prod 8501
        dev = 8001
        prod = 8501
        debug = 7501
        
    dsco update_port --debug 7600
        dev = unchanged
        prod = unchanged
        debug = 7600
```

Once you've changed the port you will need to run `dsco up` make the 
changes take effect.

Note the port is 8001.

In [38]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     35 seconds ago                1.75GB    
    foo_dev_1                     http://localhost:8001                      cca408b97e0a     Up 24 seconds                           


In [39]:
!dsco update_port 8023

Still 8001, we need to restart it.

In [40]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     37 seconds ago                1.75GB    
    foo_dev_1                     http://localhost:8001                      cca408b97e0a     Up 26 seconds                           


In [41]:
!dsco up

docker-compose up -d dev
Recreating foo_dev_1 ... 
eating foo_dev_1 ... done
up execution time: 1.7434639930725098 seconds


Now we see it has updated.

In [42]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     40 seconds ago                1.75GB    
    foo_dev_1                     http://localhost:8023                      c9769c1bbe1f     Up Less than a second                   


Put it back.

In [43]:
!dsco update_port 8001

In [44]:
!dsco up

docker-compose up -d dev
Recreating foo_dev_1 ... 
eating foo_dev_1 ... done
up execution time: 1.7722828388214111 seconds


In [45]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     43 seconds ago                1.75GB    
    foo_dev_1                     http://localhost:8001                      2d0cdc474c9b     Up Less than a second                   


## --rm

In [46]:
!dsco rm --help

usage: dsco rm [-h] [--dev] [--prod] [--debug] [--all]

optional arguments:
  -h, --help  show this help message and exit
  --dev       remove dev
  --prod      remove prod
  --debug     remove debug
  --all       remove dev, prod, and debug


`dsco rm` will by default remove the `dev` container.  
Add the appropriate flag to modify this behavior.  

In [47]:
!dsco rm

docker rm -f $(docker ps --all --filter 'label=com.docker.compose.project=foo' --filter 'label=com.docker.compose.service=dev' --format {{.Names}})
foo_dev_1


In [48]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     46 seconds ago                1.75GB    


## --rmi

In [49]:
!dsco rmi --help

usage: dsco rmi [-h] [--dev] [--prod] [--debug] [--all] [--force]

optional arguments:
  -h, --help  show this help message and exit
  --dev       remove dev
  --prod      remove prod
  --debug     remove debug
  --all       remove dev, prod, and debug
  --force     remove corresponding containers


`dsco rmi` will by default remove the `dev` image.  
Add the appropriate flag to modify this behavior.  

`dsco rmi` will fail is there is a container running that is based 
on the image you are trying to delete.

In [50]:
!dsco up
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

docker-compose up -d dev
Creating foo_dev_1 ... 
ting foo_dev_1 ... done
up execution time: 1.441547155380249 seconds
localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      
foo_dev:latest                                                               d5c29dc3707d     49 seconds ago                1.75GB    
    foo_dev_1                     http://localhost:8001                      5f64ef8e55a5     Up Less than a second                   


In [51]:
!dsco rmi

docker rmi  foo_dev
Error response from daemon: conflict: unable to remove repository reference "foo_dev" (must force) - container 5f64ef8e55a5 is using its referenced image d5c29dc3707d


You can force the removal of both with the `--force` flag.

In [52]:
!dsco rmi --all --force

docker rm -f $(docker ps --all --filter 'label=com.docker.compose.project=foo' --filter 'label=com.docker.compose.service=dev' --format {{.Names}})
foo_dev_1
docker rm -f $(docker ps --all --filter 'label=com.docker.compose.project=foo' --filter 'label=com.docker.compose.service=prod' --format {{.Names}})
"docker rm" requires at least 1 argument.
See 'docker rm --help'.

Usage:  docker rm [OPTIONS] CONTAINER [CONTAINER...]

Remove one or more containers
docker rm -f $(docker ps --all --filter 'label=com.docker.compose.project=foo' --filter 'label=com.docker.compose.service=debug' --format {{.Names}})
"docker rm" requires at least 1 argument.
See 'docker rm --help'.

Usage:  docker rm [OPTIONS] CONTAINER [CONTAINER...]

Remove one or more containers
docker rmi --force foo_dev
Untagged: foo_dev:latest
Deleted: sha256:d5c29dc3707de3eb19f8fcca7a63414d3cea707cb1f0f50de9c95dbe2d2d49c4
Deleted: sha256:36097f11cd5334ebdd9a51a6e28eab3946b8319b3aa8a48cdd4fbe0d9280aab9
Deleted: sha256:79a8cc5a675

In [53]:
# grep is just filtering the content. Generally just use 'dsco ls'
!dsco ls | egrep 'localhost|foo|NAME'

localhost                                                                                                                             
NAME                              LINK                                       ID               STATUS                        SIZE      


Finish by deleting our project directory.

In [54]:
cd /tmp

/private/tmp


In [55]:
rm -rf /tmp/dsco_overview

In [56]:
ls

AlTest1.err                             port.61201=
AlTest1.out                             port.61301=
ICRC_ROLL.lck                           port.61302=
MSS_Menulet                             port.61304=
adobegc.log                             port.61401=
com.adobe.AdobeIPCBroker.ctrl-ap186098= port.61501=
com.apple.launchd.0R9DDaYyZ6/           powerlog/
com.apple.launchd.McPXyPKx0Y/           tmux-1937012048/
com.apple.launchd.vgkMLHbKkP/           wbxtra_03252020_132958.wbt
meetingJMT.txt                          wbxtrace/
port.61100=                             webex/
